In [1]:
import findspark
findspark.init()

In [2]:
# import libraries
from pyspark import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from datetime import datetime
from pyspark.sql.functions import mean, stddev, col, log
from pyspark.sql.functions import to_date, dayofweek, to_timestamp
from pyspark.sql import types 
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DateType
from pyspark.sql.functions import year, month
from pyspark.sql.functions import dayofmonth, weekofyear
from pyspark.sql.functions import split, explode
from pyspark.sql.functions import coalesce, first, lit
from pyspark.ml.feature import Binarizer
from pyspark.ml.feature import Bucketizer
from pyspark.ml.feature import OneHotEncoder, StringIndexer, OneHotEncoderEstimator
from pyspark.sql.functions import regexp_extract, col
from pyspark.sql.functions import datediff
from pyspark.sql.functions import when

from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import LinearRegressionModel
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import GBTClassifier, RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pandas as pd

In [3]:
spark = SparkSession.builder.appName('ccpp').getOrCreate()

In [4]:
data = pd.concat(pd.read_excel('Folds5x2_pp.xlsx', sheet_name=None), ignore_index=True)
df = spark.createDataFrame(data)

In [5]:
df.count()

47840

In [6]:
df.show(5)

+-----+-----+-------+-----+------+
|   AT|    V|     AP|   RH|    PE|
+-----+-----+-------+-----+------+
|14.96|41.76|1024.07|73.17|463.26|
|25.18|62.96|1020.04|59.08|444.37|
| 5.11| 39.4|1012.16|92.14|488.56|
|20.86|57.32|1010.24|76.64|446.48|
|10.82| 37.5|1009.23|96.62| 473.9|
+-----+-----+-------+-----+------+
only showing top 5 rows



In [7]:
df.printSchema()

root
 |-- AT: double (nullable = true)
 |-- V: double (nullable = true)
 |-- AP: double (nullable = true)
 |-- RH: double (nullable = true)
 |-- PE: double (nullable = true)



In [8]:
df.select([count(when(isnan(c), c)).alias(c) for c in df.columns]).toPandas().T

,0
AT,0
V,0
AP,0
RH,0
PE,0


##### do not have NaN value

In [9]:
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).toPandas().T

,0
AT,0
V,0
AP,0
RH,0
PE,0


##### do not have null value

In [10]:
dup_rows = df.count() - df.distinct().count()
dup_rows

38313

##### remove duplicate

In [11]:
df = df.drop_duplicates()
dup_rows = df.count() - df.distinct().count()
dup_rows

0

In [12]:
df.printSchema()

root
 |-- AT: double (nullable = true)
 |-- V: double (nullable = true)
 |-- AP: double (nullable = true)
 |-- RH: double (nullable = true)
 |-- PE: double (nullable = true)



In [13]:
assembler = VectorAssembler(inputCols = ['AT', 'V', 'AP', 'RH'], outputCol = 'features')

In [14]:
output = assembler.transform(df)

In [15]:
final = output.select('features', 'PE')
final.show(5)

+--------------------+------+
|            features|    PE|
+--------------------+------+
|[24.54,60.29,1017...|447.67|
|[10.59,42.49,1009...|477.49|
|[26.7,66.56,1005....|430.21|
|[21.24,41.67,1012...|459.81|
|[27.74,74.78,1010...|436.87|
+--------------------+------+
only showing top 5 rows



In [16]:
train_data, test_data = final.randomSplit([0.75, 0.25])

In [17]:
dtc = DecisionTreeClassifier(labelCol = 'PE', featuresCol = 'features')
rfc = RandomForestClassifier(labelCol = 'PE', featuresCol = 'features')
gbt = GBTClassifier(labelCol = 'PE', featuresCol = 'features')
lr = LinearRegression(labelCol = 'PE', featuresCol = 'features')

In [18]:
lr_model  = lr.fit(train_data)

In [19]:
lr_pred = lr_model.transform(test_data)

In [20]:
lr_pred.select('PE', 'prediction').show()

+------+------------------+
|    PE|        prediction|
+------+------------------+
|486.58|480.66717630884204|
|472.16|470.68367170957004|
|458.67| 458.7200735851188|
|430.21| 437.2995217915328|
|431.83| 433.3814423152943|
| 426.2| 432.8773469852593|
| 440.3| 438.3211911369461|
|436.92| 433.3178923769423|
|483.87|477.63420912839126|
|469.74| 467.6344574584626|
|468.57| 462.6565866036146|
|460.47|463.52361380426106|
|457.98| 457.5022917337126|
|454.29|454.00948552077904|
|444.74| 448.1671600310932|
|440.22|442.07217352075764|
|441.71|  442.172353603073|
|435.84|439.51791007900647|
|437.47| 431.4003961785566|
|462.94| 469.8582685864238|
+------+------------------+
only showing top 20 rows



In [21]:
lr_model.coefficients

DenseVector([-1.9791, -0.2322, 0.059, -0.159])

In [22]:
lr_model.intercept

457.67979076888264

In [23]:
result = lr_model.evaluate(test_data)

In [24]:
result.rootMeanSquaredError

4.573974426530348

In [25]:
result.meanSquaredError

20.921242054553627

In [26]:
result.r2

0.9272192980204631

##### Linear regression works the best for this dataset